## Algorithme génétique

Les algorithmes génétiques sont des techniques de recherche stochastiques (par l'effet du hasard), inspirées par l’évolution biologique des espèces.

Dans notre cas, l'algorithme génétique est une méta-heuristique bien adapté à notre problème car il permet d'explorer efficacement un grand espace de recherche pour trouver des solutions potentiellement optimales. Pour notre problème, il peut y avoir un grand nombre de combinaisons possibles de routes, l'algorithme génétique permet donc de rechercher rapidement des solutions potentiellement proche de la solution optimale.

Un algorithme génétique consiste à faire évoluer progressivement, par générations successives, la composition de cette population, en maintenant sa taille constante : d’une génération à la suivante. Plus on avance dans les générations, plus on est sensé améliorer la fonction objectif de notre problème. Un tel résultat est obtenu en mimant les deux principaux mécanismes qui régissent l’évolution des êtres vivants : la sélection naturelle (qui détermine quels membres d’une population survivent et se reproduisent) et la reproduction (qui assure le brassage et la recombinaison des gènes parentaux, pour former des descendants aux potentialités nouvelles).<br />
Une génération est composée de $n$ individus

Dans notre cas, un individu correspond aux chemins empruntés par les k camions.<br />
Par exemple, si nous avons 3 camions qui livrent tous les client, l'individu sera décrit par les 3 chemins correspondant aux chemins des 3 camions.

### Génération de la matrice
Tout d'abord, nous devons générer une matrice connexe pondérée quelconque sur laquelle nous executerons notre algorithme génétique afin de trouver une solution convenable. Pour rappel, notre objectif est de faire en sorte que minimiser la date de retour du dernier camion au dépôt.
Le nombre de villes de la matrice correspond à la variable $\textit{num_villes}$.
Voici le code permettant de générer la matrice:

In [11]:
import sys
import random
import numpy as np
import matplotlib.pyplot as plt
import heapq

num_villes = 400

def generate_weighted_adjacency_matrix(n):
    # Générer une matrice d'adjacence vide
    adjacency_matrix = [[0] * n for _ in range(n)]

    # Générer des liens aléatoires
    for i in range(n):
        for j in range(i + 1, n):
            if np.random.random() < 0.6:  # Probabilité de lien entre deux villes
                weight = np.random.randint(1, 1000)  # Poids du lien entre deux villes
                adjacency_matrix[i][j] = weight 
                adjacency_matrix[j][i] = weight
                

    return adjacency_matrix

adj_matrix = generate_weighted_adjacency_matrix(num_villes)
print("finished")

finished


### Algorithme de Dijkstra
Avant de continuer plus loin dans l'implémentation de l'algorithme génétique, nous implémentons l'algorithme de Dijkstra. Cet algorithme permet de trouver le plus court chemin entre deux point dans un graphe connexe quelconque.<br />
Il nous permettra de faire en sorte que nos camions reviennent au dépôt lors de la génération de solutions initiales et de réparer les chemins obtenus d'une génération à une autre.

In [12]:
def dijkstra(adj_matrix, start_city, end_city):
    num_cities = len(adj_matrix)
    distances = [sys.maxsize] * num_cities
    visited = [False] * num_cities
    previous = [None] * num_cities

    distances[start_city] = 0

    for _ in range(num_cities):
        min_dist = sys.maxsize
        current_city = None

        for city in range(num_cities):
            if not visited[city] and distances[city] < min_dist:
                min_dist = distances[city]
                current_city = city

        visited[current_city] = True

        if current_city == end_city:
            break

        for neighbor in range(num_cities):
            if (
                not visited[neighbor]
                and adj_matrix[current_city][neighbor] != 0
                and distances[current_city] + adj_matrix[current_city][neighbor] < distances[neighbor]
            ):
                distances[neighbor] = distances[current_city] + adj_matrix[current_city][neighbor]
                previous[neighbor] = current_city

    if distances[end_city] == sys.maxsize:
        return "No path found."

    path = []
    current_city = end_city
    while current_city is not None:
        path.append(current_city)
        current_city = previous[current_city]

    path.reverse()
    return path, distances[end_city]

### Solutions initiales

Afin d'utiliser l'algorithme génétique, nous devons générer des solutions correctes intiales. En effet, l'objectif est de partir de solutions initiales correctes, d'y appliquer des opérateur algorithmiques (Sélection, Croisement, Mutation) dans le but de trouver une solution convenable, c'est-à-dire, assez proche de la solution optimale de tel sorte qu'on puisse la considérer comme optimale et donc viable. 
Le script ci-dessous permet de générer $\textit{numValidSol}$ solutions valides (individus) en fonction d'un nombre $\textit{NumTruck}$ de camions.

Le script fonctionne de telle sorte : 
 - A chaque ville, on regarde si il y a des villes valides encore non visitées. S'il y a des villes non visitées, on en choisit une au hasard parmit les villes non visitées et on s'y rend. Si toutes les villes voisines sont visitées, on en choisit une au hasard et on s'y rend dans le but de se rendre aux villes non visitées.

 - Une fois que toutes les villes ont été visités on utilise l'algorithme de Dijkstra pour que les camions reviennent au dépot avec le chemin le plus court possible.

Pour chaque individu, on effectue ce script. La grande part d'aléatoire permet de générer des solutions variées et donc permet d'éviter de se bloquer dès le début dans un optimum local. 

In [13]:
NumTruck = 3

population = []
truckJourneys = []
visitedCities = []

initCity = np.random.randint(1,num_villes)

numValidSol = 12

for n in range(numValidSol):
    for i in range(NumTruck):
        truckJourneys.append([initCity])
    visitedCities.append(initCity)
    print("Initial cities : ", truckJourneys)

    while len(visitedCities) < num_villes:
        for j in range(NumTruck):
            curTruck = j
            curCity = truckJourneys[curTruck][-1]
            neighborsCurCity = adj_matrix[curCity]
            CityFound = False
            # Find the indexes of non-null values in neighborsCurCity
            non_zero_indexes = np.nonzero(neighborsCurCity)[0]
            # Filter out the indexes that are in visitedCities
            non_zero_indices = np.setdiff1d(non_zero_indexes, visitedCities)
            if np.isin(non_zero_indices, visitedCities).all():
                non_zero_count = np.count_nonzero(neighborsCurCity)
                probabilities = np.where(np.array(neighborsCurCity) == 0, 0, 1 / non_zero_count)
                selectedCity = neighborsCurCity.index(np.random.choice(neighborsCurCity, p=probabilities))
                truckJourneys[curTruck].append(selectedCity)
            else:
                size_non_zero_indices = len(non_zero_indices)
                probabilities = np.zeros(len(neighborsCurCity))
                probabilities[non_zero_indices] = 1 / size_non_zero_indices
                probabilities[visitedCities] = 0            
                selectedCity = neighborsCurCity.index(np.random.choice(neighborsCurCity, p=probabilities))
                truckJourneys[curTruck].append(selectedCity)
                visitedCities.append(selectedCity)

    for j in range(len(truckJourneys)):
        result = dijkstra(adj_matrix, truckJourneys[j][-1], truckJourneys[j][0])
        path, distance = result
        truckJourneys[j] = truckJourneys[j] + path[1:]
    
    population.append(truckJourneys)
    
    truckJourneys = []
    visitedCities = []

print(population)

Initial cities :  [[175], [175], [175]]
Initial cities :  [[175], [175], [175]]
Initial cities :  [[175], [175], [175]]
Initial cities :  [[175], [175], [175]]
Initial cities :  [[175], [175], [175]]
Initial cities :  [[175], [175], [175]]
Initial cities :  [[175], [175], [175]]
Initial cities :  [[175], [175], [175]]
Initial cities :  [[175], [175], [175]]
Initial cities :  [[175], [175], [175]]
Initial cities :  [[175], [175], [175]]
Initial cities :  [[175], [175], [175]]
[[[175, 266, 76, 78, 163, 145, 208, 282, 326, 258, 98, 138, 184, 120, 357, 249, 139, 322, 235, 142, 121, 74, 179, 217, 262, 210, 14, 106, 41, 42, 339, 32, 146, 293, 0, 87, 128, 111, 10, 383, 38, 61, 173, 241, 323, 61, 165, 127, 167, 93, 118, 134, 126, 212, 240, 215, 59, 82, 318, 80, 121, 1, 248, 312, 7, 306, 194, 297, 169, 233, 130, 380, 289, 11, 226, 379, 15, 260, 301, 331, 70, 227, 3, 19, 257, 62, 278, 33, 309, 119, 195, 159, 315, 153, 332, 202, 221, 330, 193, 319, 45, 164, 298, 324, 206, 351, 107, 26, 213, 218, 

Maintenant, nous sommes prêt pour notre algorithme génétique, nous pouvons commencer à implémenter l'algorithme génétiques.

### Variables générales 

Nous commençons par initialiser les variables générales nécessaire pour notre algorithme génétique.
 - $\textit{POPULATION_SIZE}$ correspond au nombre d'individus d'une population et donc au nombre d'individus initaux qu'on créé
 - $\textit{NUM_GENERATIONS}$ correspond au nombre de générations que l'on va créé et donc au nombr ed'itérations du programme.
 - $\textit{MUTATION_RATE}$ correspond au taux de mutation, cette variable est extrêmement importante. L'objectif est d'avoir le taux parfait de tel sorte à explorer les optiumum locaux suffisemment afin de savoir si c'est un optimum local ou un optimum que l'on peut considérer comme global. Un taux de mutation trop faible empêche que visiter tout les optimums locaux puisqu'il s'enferme dans un seul, un taux de mutation trop élevé empêche de trouver un optimum global car on ne s'intéresse à aucun optimum local. 

In [14]:
# Constants
POPULATION_SIZE = numValidSol
NUM_GENERATIONS = 100
MUTATION_RATE = 0.1

### Calcul de la distance

Afin de pouvoir savoir quels sont les solutions qui minimisent au mieux la fonction objectif, nous devons être capable de mesurer les distances parcourues par les camions pour chaque individu.<br />
La fonction $\textit{calculate_fitness}$ s'occupe de mesurer la distance parcourue par chaque camion d'un individu et calcul la difference entre la valeur maximale et la valeur minimale.<br />
Calculer la difference permet vérifier la condition initiale suivante : "Minimiser la date de retour du dernier camion au dépôt". On cherche donc à trouver la différence la plus petite possible. <br />
En effet, si la différence vaut 0, cela veut dire que la charge de livraison est parfaitement répartie entre les camions et donc que le temps que met le dernier camion à revenir au dépôt est la plus petite possible (étant donné qu'il revient en même temps que le premier et tous les autres camions)

In [15]:
# Function to calculate the fitness of an individual
def calculate_fitness(individual):
    distances = [0] * len(individual)
    for i, path in enumerate(individual):
        path_distance = 0
        for j in range(len(path) - 1):
            start_city = path[j]
            end_city = path[j + 1]
            path_distance += adj_matrix[start_city][end_city]
        distances[i] = path_distance
    difference = max(distances) - min(distances)
    return difference

for individual in population:
    fitness = calculate_fitness(individual)
    print(fitness)

8794
3760
6945
4823
2068
5469
3458
11072
2075
4186
4375
5385


### Croisement

Le croisement est une opération nécessaire dans l'algorithme génétique. Celui-ci a pour but de mélanger le chemin $i$ d'un individu avec le chemin $i$ d'un autre individu.<br />
L'endroit de la séparation $\textit{cut_point}$ est choisi aléatoirement.<br />
Par exemple, si nous prenons comme $\textit{parent1}$ un individu dont le chemin 1 passe par les villes $\color{red}{3\rightarrow1\rightarrow4\rightarrow5\rightarrow2\rightarrow6\rightarrow3}$.<br />
Et comme $\textit{parent2}$ un autre individu dont le chemin 1 passe par les villes $\color{blue}{3\rightarrow5\rightarrow8\rightarrow9\rightarrow7\rightarrow2\rightarrow3}$<br /> 
Et que l'endroit de la séparation $\textit{cut_point} = 3$.<br />
Le chemin 1 de l'enfant entre le parent1 et le parent2 vaudra : 
$\color{red}{3\rightarrow1\rightarrow4}\color{blue}{\rightarrow9\rightarrow7\rightarrow2\rightarrow3}$

Bien sûr, il n'est pas assuré qu'il existe un lien direct entre la vile $4$ et la ville $9$. C'est pour cela que nous appliquons des réparations aux chemin afin d'avoir des chemins correctes. Nous expliquerons les réparations plus loin dans le document.

In [16]:
# Function to perform crossover between two parents
def crossover(parent1, parent2):
    child = [[] for _ in range(len(parent1))]
    for i in range(len(parent1)):
        cut_point = np.random.randint(0, len(parent1[i]) - 1)
        child[i] = parent1[i][:cut_point] + parent2[i][cut_point:]
        child[i] = repair_non_neighboring_cities(child[i], cut_point)

    child = repair_missing_cities(child)  # Repair the child to ensure it visits all cities
    return child

### Mutation

Comme expliquer précédemment, l'opérateur de mutation nous permet d'éviter d'être coincé dans un optimum local.

Lorsqu'une mutation est effectuée, nous interchangeons la position entre deux villes choisies au hasard dans un chemin.<br />
Par exemple, on reprend le chemin d'un individu utilisé plus haut : $\color{red}{3\rightarrow1\rightarrow4\rightarrow5\rightarrow2\rightarrow6\rightarrow3}$.<br />
On choisit $2$ indexes de la liste au hasard et on les interchange.<br />
Si on obtient l'indexe $1$ et l'index $5$, le chemin muté sera : $\color{red}{3\rightarrow}\color{green}{6}\color{red}{\rightarrow4\rightarrow5\rightarrow2\rightarrow}\color{green}{1}\color{red}{\rightarrow3}$

Comme pour l'opérateur de croisement, il n'est pas assuré que les villes $3-6$, $6-4$, $2-1$ et $1-3$ soient reliés. Si elles ne le sont pas, nous devons apporter des réparation à notre chemin pour avoir un chemin correct.<br />

In [17]:
# Function to perform mutation on an individual
def mutate(individual):
    for i in range(len(individual)):
        if np.random.random() < MUTATION_RATE:
            swap_indices = random.sample(range(1, len(individual[i]) - 1), 2)
            individual[i][swap_indices[0]], individual[i][swap_indices[1]] = \
                individual[i][swap_indices[1]], individual[i][swap_indices[0]]
            for index in swap_indices:
                repair_non_neighboring_cities(individual[i], index)
                if index > 0:
                    repair_non_neighboring_cities(individual[i], index-1)
    return individual

### Réparation des erreurs

Comme indiqué ci-dessus, lors des opérations de Croisement et de Mutation, il y a de fortes chances que deux villes soient indiquées comme voisines dans le chemin, hors elles ne le sont pas.<br />
C'est pour cela que nous devons appliquer des réparations sur la chaine pour qu'elle soit correcte. 
Etant donné que nous savons où les erreurs peuvent être situées nous indiquons à la fonction les endroits où effectuer la vérification.<br />
Dans le cas où les deux villes ne sont pas voisines, nous effectuons l'algorithme de Dijkstra afin de connaitre le chemin le plus court entre les deux villes, puis nous ajoutons ce chemin, qui lie les deux villes, entre les deux viles dans le chemin initial.

In [18]:
def repair_non_neighboring_cities(path, index):
    if index+1 <= len(path)-1:
        city1 = path[index]
        city2 = path[index+1]
        if city1 == city2:
            del path[index]
        else:
            adj_value = adj_matrix[city1][city2]
            if adj_value == 0:
                result = dijkstra(adj_matrix, city1, city2)
                path_to_add, distance = result
                path[index+1:index+1] = path_to_add[1:-1]
    return path

De plus, il est possible qu'après les opérations de croisement, un individu se retrouve avec des villes manquantes. Or, nous devons absulment passer par par toutes les villes (livrer tous les clients), c'est pour cela que nous avons implémenter une autre fonction qui d'ocuupe de vérifier si toutes les villes sont visitées.<br />
Si certaines villes sont manquantes, nous les ajoutons dans notre chemin de manière intelligente.<br />
Tout d'abord, on cherche un emplacement où on pourrait insérer notre ville manquante, de telle sorte que la ville précédente et la ville suivantes soient des villes voisines à la ville manquante.<br />
Si on ne trouve pas d'emplacement satisfaisant, on insère notre ville après une ville voisine et on effectue l'algorithme de Dijkstra avec la ville suivante, afin d'insérer le chemin le plus court entre la ville insérée et la ville suivante. 

In [19]:
def repair_missing_cities(individual):
    visited_cities = set(city for path in individual for city in path)
    missing_cities = set(range(len(adj_matrix))) - visited_cities
    while len(missing_cities) > 0:
        missing_city = missing_cities.pop()
        for path in individual:
            for i in range(len(path)-1):
                if adj_matrix[missing_city][i] != 0 and adj_matrix[missing_city][i+1] != 0:
                    path.insert(i+1, missing_city)
                    missing_city = -1
                    break
            if missing_city == -1:
                break
        if missing_city != -1:
            neighborCities = [i for i, val in enumerate(adj_matrix[missing_city]) if val != 0 and i not in missing_cities]
            neighborCity = np.random.choice(neighborCities)
            for path in individual:
                try:
                    indexneighborCity = path.index(neighborCity)
                    path.insert(indexneighborCity+1, missing_city)
                    result = dijkstra(adj_matrix, indexneighborCity+1, indexneighborCity+2)
                    path_to_add, distance = result
                    index_to_add = indexneighborCity+2
                    path[index_to_add:index_to_add] = path_to_add[1:-1]
                except:
                    pass
    return individual

### Sélection et exécution de l'algorithme

A présent, nous avons toutes les fonctions nécessaires pour exécuter notre algorithme génétique.<br />
Nous commençons par afficher le meilleur individu dans la population actuelle.<br />
Ensuite, on sélectionne $6$ individu parents dans la population pour faire $6$ individus enfants. Pour choisir les individus, nous choisissons $5$ individus dans la population parent puis nous sélectionnons l'individu qui minimise au plus la fonction objectif parmis ces $5$ individus.<br />
Une fois la sélection effectuée, nous appliquons les opération de croisements et de mutations afin de créer une génération enfant.<br />
Une fois la génération enfant créée, on garde uniquement les $12$ meilleurs individus de la génération parent et la génération enfant qui satisfaisent au plus la fonction objectif.<br />
On effectue ces opérations $\textit{NUM_GENERATIONS}$ fois, au bout des ces $\textit{NUM_GENERATIONS}$ fois, on considère que la solution trouvée est optimale.

In [20]:
# Main genetic algorithm loop
for generation in range(NUM_GENERATIONS):
    # Calculate fitness for each individual
    fitness_scores = [calculate_fitness(individual) for individual in population]
    
    # Find the best individual in the population
    best_fitness = min(fitness_scores)
    best_individual = population[fitness_scores.index(best_fitness)]
    
    print(f"Generation {generation + 1}: Best Distance = {best_fitness}")
    
    # Select parents for crossover using tournament selection
    parents = []
    for _ in range(POPULATION_SIZE // 2):
        tournament = random.sample(range(POPULATION_SIZE), 5)
        tournament_fitness = [fitness_scores[i] for i in tournament]
        winner = tournament[tournament_fitness.index(min(tournament_fitness))]
        parents.append(population[winner])
    
    # Create new population through crossover and mutation
    new_population = []
    for i in range(0, len(parents), 2):
        parent1 = parents[i]
        parent2 = parents[i + 1]
        child1 = crossover(parent1, parent2)
        child2 = crossover(parent2, parent1)
        new_population.extend([mutate(child1), mutate(child2)])
        
    population.extend(new_population)
    fitness_scores.extend([calculate_fitness(individual) for individual in new_population])

    
    # Combine the elements of lists population and fitness_scores
    combined = zip(population, fitness_scores)

    # Sort the pairs based on the values in list fitness_scores
    sorted_pairs = sorted(combined, key=lambda x: x[1])

    # Extract the desired values
    population = [pair[0] for pair in sorted_pairs[:POPULATION_SIZE]]
    fitness_scores = [pair[1] for pair in sorted_pairs[:POPULATION_SIZE]]
    
    
# Print the best solution
best_fitness = min(fitness_scores)
best_individual = population[fitness_scores.index(best_fitness)]
print(f"\nBest Solution: {best_individual}")
print(f"Best Distance: {best_fitness}")

Generation 1: Best Distance = 2068
Generation 2: Best Distance = 2068
Generation 3: Best Distance = 2068
Generation 4: Best Distance = 1674
Generation 5: Best Distance = 1674
Generation 6: Best Distance = 1674
Generation 7: Best Distance = 1674
Generation 8: Best Distance = 1674
Generation 9: Best Distance = 1612
Generation 10: Best Distance = 1612
Generation 11: Best Distance = 1612
Generation 12: Best Distance = 1612
Generation 13: Best Distance = 561
Generation 14: Best Distance = 561
Generation 15: Best Distance = 561
Generation 16: Best Distance = 561
Generation 17: Best Distance = 438
Generation 18: Best Distance = 438
Generation 19: Best Distance = 438
Generation 20: Best Distance = 438
Generation 21: Best Distance = 438
Generation 22: Best Distance = 438
Generation 23: Best Distance = 438
Generation 24: Best Distance = 438
Generation 25: Best Distance = 438
Generation 26: Best Distance = 438
Generation 27: Best Distance = 438
Generation 28: Best Distance = 438
Generation 29: Be